In [ ]:
import os 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time 
import pickle
import warnings

from utils import *

from sensor_params import *

from models.training import *

from models.unet1D_regressor import *
from models.unet1D_regressor import get_model as get_unet1D

from models.unet1D_DIL_regressor import *
from models.unet1D_DIL_regressor import get_model as get_DIL_unet1D

from models.LSTMVanilla import *
from models.LSTMVanilla import get_model as get_LSTM

from models.unet1D_LSTM_regressor import *
from models.unet1D_LSTM_regressor import get_model as get_unet1DLSTM

from models.unet1D_nonCompres_regressor import *
from models.unet1D_nonCompres_regressor import get_model as get_unet1D_nonCompres


# Dataset path 
DATABASE_PATH = r"C:\Users\aralmeida\OneDrive - Universidad de Las Palmas de Gran Canaria\Doctorado\Bases de datos\WARIFA\Mikael T1DM"

filename = "MIKAEL_data.json"

parent_directory = r"C:\Users\aralmeida\Downloads"
experiments_folder = r"\T1DM_pred_experiments" 

In [ ]:
# Load data from sgv_data. If executed, comment the line and load the data from the pickle file
# data , basal_data_dict, blood_pressure_dict, bolus_data_dict, exercise_data_dict, carbohydrates_data_dict, pump_events_data_dict, sgv_data_dict, sleep_data_dict, smgb_data_dict, steps_data_dict, weight_dict  = extract_Mikael_data(DATABASE_PATH, filename, ONLY_CGM = True)
with open(DATABASE_PATH+'\CGM.pk1', 'rb') as f:
    sgv_data_dict = pickle.load(f)
os.chdir(parent_directory)

In [ ]:
N = 144
tau = 1
PH = 30
pred_steps = 6
name = '1DUNET-non-compres_himar-rep'

In [ ]:
X, Y, X_times, Y_times = get_CGM_X_Y(sgv_data_dict, sensor_Mikael, N, tau, PH, experiments_folder, plot=False, verbose = 0)

In [ ]:
# Convert np.arrays to float32 to convert them to Tensorflow tensors
X = X.astype(np.float32)
Y = Y.astype(np.float32)

In [ ]:
# Simple partition (imitating Himar work for comparison): X_train until 30/05/2021 and X_test from 31/05/2021
# Samples that include two days are removed
X_train = X[np.where(X_times[:,N-1] <= pd.to_datetime('2021-05-31 00:00:00'))[0]]
Y_train = Y[np.where((Y_times[:,0] < pd.to_datetime('2021-05-30 23:59:59')))[0]]
X_test = X[np.where((X_times[:,N-1] > pd.to_datetime('2021-06-01 00:00:00')))[0]]
Y_test = Y[np.where((Y_times[:,0] > pd.to_datetime('2021-06-01 00:00:00')))[0]]

print("X_train shape: ",str(X_train.shape))
print("Y_train shape: ",str(Y_train.shape))
print("X_test shape: ",str(X_test.shape))
print("Y_test shape: ",str(Y_test.shape))

In [ ]:
# Downsampling to have the same number of samples for each range (normal, hyperglycemia, hypoglycemia) in the Y vector
severe_hypo_th = 54
hypo_th = 70
hyper_th = 180
severe_hyper_th = 250

# count number of Y samples > hyper 
normal_count = np.count_nonzero((Y_train >= hypo_th) & (Y_train <= hyper_th))
print("Number of normal range points to train: ", normal_count)
hyper_count = np.count_nonzero(Y_train > hyper_th)
print("Number of hyperglycemia points to train: ", hyper_count)
hypo_count = np.count_nonzero(Y_train < hypo_th)
print("Number of hypoglycemia points to train: ", hypo_count)

# Percentage of samples in each range 
print("Percentage of samples in the normal range: ", round(normal_count / Y_train.shape[0]*100, 2))
print("Percentage of samples in the hyperglycamia range: ", round(hyper_count / Y_train.shape[0]*100, 2))
print("Percentage of samples in the hypoglycamia range: ", round(hypo_count / Y_train.shape[0]*100, 2))

In [ ]:
# Indices of the samples in each range
normal_Y_idxs = np.where((Y_train >= hypo_th) & (Y_train <= hyper_th))[0]
hypo_Y_idxs = np.where(Y_train < hypo_th)[0]
hyper_Y_idxs = np.where(Y_train > hyper_th)[0]

In [ ]:
# # Set a random seed for reproducibility
# np.random.seed(42)

# # Undersampling to randomly select idx from the normal values with a random seed
# normal_Y_idxs = np.random.choice(normal_Y_idxs, size=hyper_count*2, replace=False)

# # X and Y train undersampled with the obtained indices
# X_train = np.concatenate((X_train[normal_Y_idxs], X_train[hypo_Y_idxs], X_train[hyper_Y_idxs]))
# Y_train = np.concatenate((Y_train[normal_Y_idxs], Y_train[hypo_Y_idxs], Y_train[hyper_Y_idxs]))

# # Concatenate X_train and Y_train to shuffle them in the same way
# XY_train = np.concatenate((X_train, Y_train), axis=1)

In [ ]:
print(X_train[11,143])
print(Y_train[0,0])

In [ ]:
# print(XY_train[11,143])
# print(XY_train[0,144])

In [ ]:
# # Shuffle X_Y_train
# np.random.shuffle(XY_train)

# # Split X and Y train again
# X_train = XY_train[:,:-1]
# Y_train = XY_train[:,-1]

In [ ]:
# Parameters for training
# Best parameters for the model N 48
lr = 0.0001
batch_size = 2
epochs = 10

kernel_size = 3

In [ ]:
# Get model instance
model =  get_unet1D(N=N,
                input_features = NUMBER_OF_INPUT_SIGNALS,
                tau=tau,
                kernel_size=kernel_size,
                predicted_points=6)

# model =  get_unet1D_nonCompres(N=N,
#                 input_features = NUMBER_OF_INPUT_SIGNALS,
#                 tau=tau,
#                 kernel_size=kernel_size,
#                 predicted_points=1)

# model =  get_DIL_unet1D(N=N,
#                 input_features = NUMBER_OF_INPUT_SIGNALS,
#                 tau=tau,
#                 kernel_size=kernel_size,
#                 dilation_rate=1,  
#                 predicted_points=1)

# model =  get_LSTM(N=int(N),
#                 input_features = NUMBER_OF_INPUT_SIGNALS,
#                 tau=tau,
#                 kernel_size=kernel_size,
#                 predicted_points=1)

# model =  get_unet1DLSTM(N=N,
#                 input_features = NUMBER_OF_INPUT_SIGNALS,
#                 tau=tau,
#                 kernel_size=kernel_size,
#                 predicted_points=1)

In [ ]:
train_model(model,
            X = X_train,
            Y = Y_train,
            N = N,
            tau = tau,
            kernel_size = kernel_size,
            predicted_points = sensor_Mikael["PREDICTED_POINTS"],
            sensor_name = sensor_Mikael,
            epochs = epochs,
            batch_size = batch_size,
            lr = lr,
            verbose = 1, 
            fold = name)

In [ ]:
model_evaluation(name, X_test, Y_test)